## 머신러닝을 이용한 언어 감지 서비스 구축

### 1. 연구 목표 설정

- 유사서비스 : 파파고, 구글 번역
- 개요
    - 번역 서비스중 언어 감지 파트는 머신러닝의 지도학습법 중 분류를 사용하겠다.
    - 알파벳을 사용하는 영어권에서는 알파벳 언어별로 알파벳의 사용 빈도가 다르다.
- 조건
    - 비 영어권은 개별 방법론(완성형(utf-8), 조합형(euc-kr) 코드를 이용하여 판단) 배제
    - 임시값(100byte) 이내 문자열을 배제, 임시값의 임계값은 변경될수 있다.
    - 번역서비스는 딥러닝의 RNN을 활용하여 처리하는데 여기서는 배제, 단, 파파고 API를 활용하여 유사하게 구현
    - 서비스가 오픈하고 데이터가 축적되면 모델을 갱신(언어는 진화하니까) 모델을 다시 학습하고 교체를 진행하는데 원활하게 수행되겠금 처리(전략). 일단 여기서는 데이터 축적

|No|단계|내용|
|:---:|:---|:---|
|1|연구 목표 설정|- 웹서비스<br>- 사용자가 입력한 텍스트를 에측하여 어떤 언어인지 판독한다(영어권,알파벳 사용국가)<br>- 머신러닝의 지도학습-분류를 사용하겠다<br>- 파이프라인구축, 하이퍼파라미터튜닝을 이용한 최적화 부분은 제외<br>- 정량적인 목표치는 생략(평가배제)<br>- 임시값(100byte) 이내 문자열을 배제<br>- 논문을 통한 주장의 근거를 체크|
|2|데이터 획득/수집|- 실전:다양한 텍스트를 수집, 위키피디아, 법률, 소설등등<br>- 구현: 제공데이터를 사용(법령/대본/소설등)|
|3|데이터 준비/통찰/전처리|- 알파벳을 제외한 모든 문자 제거(전처리, 정규식)<br>- 텍스트를 알파벳의 출현 빈도로 계산한다(문자계산, 데이터의 수치화)<br>- 데이터는 훈련 데이터(훈련(50), 검증(25))와 테스트 데이터(25)로 나눈다 (훈련:테스트=75:25) 황금비율, 단 바뀔수 있다|
|4|데이터 탐색/통찰/시각화|- 논문의 주장을 증명<br>- 영어권 언어별로 알파벳 출현 빈도가 다르다는 명제를 증명/확인<br>- EDA 분석(시각화)를 이용하여 확인. 선형차트, 바차트 등을 활용|
|5|데이터 모델링 및 모델 구축|- 알고리즘 선정<br>- 학습데이터/테스트데이터 준비<br>- 학습<br>- 예측<br>- 성능평가(학습법,하위 카테고리 까지 검토 평가)<br>- 파이프라인구축을 통하여 알고리즘 체인을 적용, 최적의 알고리즘 조합을 찾는다<br>- 연구 목표에 도착할때까지 반복|
|6|시스템 통합|- 모델 덤프(학습된 알고리즘을 파일로 덤프)<br>- 웹 서비스 구축(flask 간단하게 구성)<br>- 서비스 구축<br>- 모델의 업그레이드를 위한 시스템 추가<br>- 선순화구조를 위한 번역 요청 데이터의 로그처리->배치학습, 온라인 학습등으로 연결되어 완성|

### 2. 데이터 획득
- 실전:다양한 텍스트를 수집, 위키피디아, 법률, 소설등등
    - 라이브러리 : request(get), BS4
    - 사이트 : https://언어코드.wikipedia.org/wiki/검색어
        - 예) https://en.wikipedia.org/wiki/Bong_Joon-ho
- 구현: 제공데이터를 사용(법령/대본/소설등)

In [1]:
#  모듈 가져오기
import urllib.request as req
from bs4 import BeautifulSoup


In [2]:
# 데이터 획득 사이트 규격화
target_site = 'https://{na_code}.wikipedia.org/wiki/{keyword}'.format( na_code='en', keyword='Bong_Joon-ho' )

target_site

'https://en.wikipedia.org/wiki/Bong_Joon-ho'

In [3]:
# 요청 및 SOUP 생성(DOM 트리)
# html5lib 파서 사용 이유: 대량의 html을 파싱하기 위해 안전성 고려
soup = BeautifulSoup( req.urlopen(target_site), 'html5lib' )

In [4]:
# 데이터 추출
# css selector : #mw-content-text p
tmp = soup.select('#mw-content-text p')#mw-content-text
len(tmp), type(tmp)

(22, bs4.element.ResultSet)

In [5]:
# p 밑에 있는 모든 텍스트를 리스트에 모아 둔다 => 멤버수가 => 22개

for p in soup.select('#mw-content-text p'):
    print(p.text)
    



Bong Joon-ho (Korean: 봉준호, Korean pronunciation: [poːŋ tɕuːnho → poːŋdʑunɦo]; born September 14, 1969) is a South Korean film director and screenwriter. He garnered international acclaim for his second feature film Memories of Murder (2003), before achieving commercial success with his subsequent films The Host (2006) and Snowpiercer (2013), both of which are among the highest-grossing films of all time in South Korea.[1] 

Two of his films have screened in competition at the Cannes Film Festival—Okja, which premiered at the 2017 Cannes Film Festival, and Parasite, which won the Palme d'Or at the 2019 Cannes Film Festival.[2] He became the first Korean director to win the Palme d'Or.[3] Parasite also won Best Foreign Language Film at the 77th Golden Globe Awards, with Bong nominated for Best Director and Best Screenplay for his work.[4]

In 2017, Metacritic ranked Bong sixteenth on its list of the 25 best film directors of the 21st century.[5] His films feature timely social themes, 

In [6]:
# 리스트 생성
texts = list() #동적 구성
for p in tmp:
    # 멤버 추가
    texts.append(p.text)
    #print( type(p.text), p.text )
len(texts), texts[:2]

(22,
 ['\n',
  'Bong Joon-ho (Korean:\xa0봉준호, Korean pronunciation:\xa0[poːŋ tɕuːnho → poːŋdʑunɦo]; born September 14, 1969) is a South Korean film director and screenwriter. He garnered international acclaim for his second feature film Memories of Murder (2003), before achieving commercial success with his subsequent films The Host (2006) and Snowpiercer (2013), both of which are among the highest-grossing films of all time in South Korea.[1] \n'])

In [7]:
# 리스트 내포
texts = [ p.text for p in tmp ]
len(texts), texts[-1:] #맨마지막 행

(22, [' Media related to Bong Joon-ho at Wikimedia Commons\n'])

In [8]:
# 수집한 데이터를 한개의 텍스트 덩어리로 획득
# a = list('helloworld')
# ''.join(a)

str_txt = ' '.join(texts)
len(str_txt), str_txt[:100]

(13266,
 '\n Bong Joon-ho (Korean:\xa0봉준호, Korean pronunciation:\xa0[poːŋ tɕuːnho → poːŋdʑunɦo]; born September 14, 1')

In [9]:
# 정규식을 활용하여 알파벳만 남긴다.
import re

In [10]:
p = re.compile('[^a-zA-Z]*')

In [11]:
tmp = p.sub( '' , str_txt )
#전부 소문자로 처리
len(tmp), tmp.lower()

(10215,
 'bongjoonhokoreankoreanpronunciationpotunhopodunobornseptemberisasouthkoreanfilmdirectorandscreenwriterhegarneredinternationalacclaimforhissecondfeaturefilmmemoriesofmurderbeforeachievingcommercialsuccesswithhissubsequentfilmsthehostandsnowpiercerbothofwhichareamongthehighestgrossingfilmsofalltimeinsouthkoreatwoofhisfilmshavescreenedincompetitionatthecannesfilmfestivalokjawhichpremieredatthecannesfilmfestivalandparasitewhichwonthepalmedoratthecannesfilmfestivalhebecamethefirstkoreandirectortowinthepalmedorparasitealsowonbestforeignlanguagefilmatthethgoldenglobeawardswithbongnominatedforbestdirectorandbestscreenplayforhisworkinmetacriticrankedbongsixteenthonitslistofthebestfilmdirectorsofthestcenturyhisfilmsfeaturetimelysocialthemesgenremixingblackhumorandsuddenmoodshiftsbongjoonhowasbornindaeguinanddecidedtobecomeafilmmakerwhileinmiddleschoolbongjoonhohadahighlyintellectualupbringinghisfatherbongsanggyunisagraphicdesignerandhismaternalgrandfatherparktaewonwasanotedauthorfamous

### 3. 데이터 준비
- 알파벳을 제외한 모든 문자 제거(전처리, 정규식)
    - 편의상 앞단계에서 같이 병행 처리했다.
    - 제공 데이터에서는 여기서 실제 처리하겠다.
- 텍스트를 알파벳의 출현 빈도로 계산한다(문자계산, 데이터의 수치화)
- 데이터는 훈련 데이터(훈련(50), 검증(25))와 테스트 데이터(25)로 나눈다 (훈련:테스트=75:25) 황금비율, 단 바뀔수 있다

In [12]:
# 데이터를 다 읽어 들인다 => 파일 목록이 필요
# glob 특정 패턴을 부여해서 특정 위치의 파일 목록을 가져온다.
import os.path, glob


In [13]:
file_list = glob.glob( './data/train/*.txt')
len(file_list), type(file_list), file_list[:2]

(20, list, ['./data/train\\en-1.txt', './data/train\\en-2.txt'])

In [14]:
# 파일을 읽어서 알파벳별 빈도계산, 그런 데이터가 en 혹은 fr등 이 데이터가 어떤 언어인지
# (레이블, 정답)까지 데이터화 한다.
# 1. 파일명 획득
fName = file_list[0]
fName

'./data/train\\en-1.txt'

In [15]:
#2. 파일명 획득
name = os.path.basename( fName )
name

'en-1.txt'

In [16]:
#3. 파일명에는 정답(레이블)이 들어 있다. 이것을 추출
# 확장성을 고려하여 정규식으로 처리

p = re.compile('^[a-z]{2,}')
if p.match( name ):
    lang = p.match( name ).group()
    p.match( name ).group()
else:
    print('일치하지 않음')
lang

'en'

In [19]:
file_list

['./data/train\\en-1.txt',
 './data/train\\en-2.txt',
 './data/train\\en-3.txt',
 './data/train\\en-4.txt',
 './data/train\\en-5.txt',
 './data/train\\fr-10.txt',
 './data/train\\fr-6.txt',
 './data/train\\fr-7.txt',
 './data/train\\fr-8.txt',
 './data/train\\fr-9.txt',
 './data/train\\id-11.txt',
 './data/train\\id-12.txt',
 './data/train\\id-13.txt',
 './data/train\\id-14.txt',
 './data/train\\id-15.txt',
 './data/train\\tl-16.txt',
 './data/train\\tl-17.txt',
 './data/train\\tl-18.txt',
 './data/train\\tl-19.txt',
 './data/train\\tl-20.txt']

In [18]:
# 4. 알파벳 빈도 계산
# 4-1. 파일을 읽는다
# 4-2. 알파벳만 남긴다(정규식으로 전처리)
# 4-3. 소문자로 처리
# 4-4. 알파벳 별로 카운트 계산=>효율적인 로직, 방법이 필요

In [20]:
file_list1 = glob.glob( './data/test/*.txt')
len(file_list1), type(file_list1), file_list1[:5]

(8,
 list,
 ['./data/test\\en-1.txt',
  './data/test\\en-2.txt',
  './data/test\\fr-3.txt',
  './data/test\\fr-4.txt',
  './data/test\\id-5.txt'])

### 4. 데이터 탐색

### 6. 시스템 통합